拉普拉斯机制

差分隐私一般用于回复特定的问询。现在考虑一个针对人口普查数据的问询。首先不使用差分隐私。

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
adult = pd.read_csv("adult_with_pii.csv")

"数据集中有多少个体的年龄大于等于40岁？"

In [3]:
adult[adult['Age'] >= 40].shape[0]

14237

使这个问询满足差分隐私的最简单方法是：在回复结果上增加随机噪声。这里的关键挑战是：既需要增加足够大的噪声，使问询满足差分隐私，但噪声又不能加得太多，否则问询结果就无意义了。为了简化这一过程，差分隐私领域的学者提出了一些基础机制。这些基础机制具体描述了应该增加何种类型的噪声，以及噪声量应该有多大。最典型的基础机制是拉普拉斯机制（Laplace Mechanism）

In [6]:
sensitivity = 1
epsilon = 0.1

adult[adult['Age'] >= 40].shape[0] + np.random.laplace(loc=0, scale=sensitivity/epsilon)

14243.09814275089

虽然每次代码的输出结果都会发生变化，但在大多数情况下，输出的结果都与真实结果（14,235）很接近，输出结果的可用性相对较高。

需要多大的噪声？

我们如何知道拉普拉斯机制是否已经增加了足够的噪声，可以阻止攻击者对数据集中的个体实施重标识攻击？可以先尝试自己来实施攻击。构造一个恶意的计数问询，专门用于确定凯莉·特鲁斯洛夫的收入是否大于$50k。

In [7]:
karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0]

1

此回复结果给出了凯莉所在数据行的收入值，显然侵犯了凯莉的隐私。由于我们知道如何应用拉普拉斯机制使计数问询满足差分隐私，我们可以这样回复问询：

In [10]:
sensitivity = 1
epsilon = 0.1

karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0] + \
  np.random.laplace(loc=0, scale=sensitivity/epsilon)

2.7518313074038887

真实结果是0还是1呢？因为增加的噪声比较大，我们已经无法可靠地判断真实结果是什么了。这就是差分隐私要实现的目的：哪怕可以判定出此问询是恶意的，我们也不会拒绝回复问询。相反，我们会增加足够大的噪声，使恶意问询的回复结果对攻击者来说变得毫无用处。